In [1]:
import pandas as pd
import os 

#Data preprocessing 

In [2]:
files = os.listdir()
files

['test_preprocessed.parquet',
 'training_preprocessed.csv',
 '.DS_Store',
 'test.csv',
 'test_preprocessed.csv',
 'training_preprocessed.parquet',
 'preprocessed_data.zip',
 'final_project_models.ipynb',
 'finalproject.ipynb',
 'train.csv',
 '.ipynb_checkpoints',
 'transactions.csv',
 'Archiwum.zip',
 'oil.csv',
 'holidays_events.csv',
 'results.csv',
 'sample_submission.csv',
 'stores.csv']

In [3]:
test = pd.read_csv('test.csv',index_col=0)
test.head()

,date,store_nbr,family,onpromotion
id,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0
3000889,2017-08-16,1,BABY CARE,0
3000890,2017-08-16,1,BEAUTY,2
3000891,2017-08-16,1,BEVERAGES,20
3000892,2017-08-16,1,BOOKS,0


In [4]:
train = pd.read_csv('train.csv',index_col=0)
train.head()

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0
3,2013-01-01,1,BEVERAGES,0.0,0
4,2013-01-01,1,BOOKS,0.0,0


In [5]:
oil = pd.read_csv('oil.csv',index_col=0)
oil.head()
#merged = train.join(oil.fillna(method='bfill'),on='date')
merged = test.join(oil.fillna(method='bfill'),on='date')
merged.head()

,date,store_nbr,family,onpromotion,dcoilwtico
id,,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0,46.8
3000889,2017-08-16,1,BABY CARE,0,46.8
3000890,2017-08-16,1,BEAUTY,2,46.8
3000891,2017-08-16,1,BEVERAGES,20,46.8
3000892,2017-08-16,1,BOOKS,0,46.8


In [6]:
stores = pd.read_csv('stores.csv',index_col=0)
stores.head()
merged = merged.join(stores,on='store_nbr')

In [7]:
holidays = pd.read_csv('holidays_events.csv')
holidays_local = holidays[holidays['locale']=='Local']
holidays_local['city'] = holidays_local['locale_name']
holidays_regional = holidays[holidays['locale']=='Regional']
holidays_national = holidays[holidays['locale']=='National']

/var/folders/r0/qwzhklln27zbg3y3p6hy60sw0000gn/T/ipykernel_44543/3268219110.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holidays_local['city'] = holidays_local['locale_name']


In [8]:
holidays_local_merged = pd.merge(merged, holidays_local,  how='left', left_on=['date','city'], right_on = ['date','locale_name'])
holidays_regional_merged = pd.merge(merged, holidays_regional,  how='left', left_on=['date','state'], right_on = ['date','locale_name'])
holidays_national_merged = pd.merge(merged, holidays_national,  how='left', left_on=['date'], right_on = ['date'])


In [9]:
final_df = pd.DataFrame()
final_df = merged.copy()
#final_df = merged.copy()
#final_df.index = holidays_local_merged.index
#final_df['id'] = holidays_local_merged['id']
#final_df['store_nbr'] = holidays_local_merged['store_nbr']
#final_df['family'] = holidays_local_merged['family']
#final_df['onpromotion'] = holidays_local_merged['onpromotion']
#final_df['dcoilwtico'] = holidays_local_merged['dcoilwtico']
#final_df['city'] = merged['city']
#final_df['state'] = merged['state']
#final_df['type'] = holidays_local_merged['type_x']
#final_df['cluster'] = holidays_local_merged['cluster']
final_df

,date,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster
id,,,,,,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,Quito,Pichincha,D,13
3000889,2017-08-16,1,BABY CARE,0,46.80,Quito,Pichincha,D,13
3000890,2017-08-16,1,BEAUTY,2,46.80,Quito,Pichincha,D,13
3000891,2017-08-16,1,BEVERAGES,20,46.80,Quito,Pichincha,D,13
3000892,2017-08-16,1,BOOKS,0,46.80,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...
3029395,2017-08-31,9,POULTRY,1,47.26,Quito,Pichincha,B,6
3029396,2017-08-31,9,PREPARED FOODS,0,47.26,Quito,Pichincha,B,6
3029397,2017-08-31,9,PRODUCE,1,47.26,Quito,Pichincha,B,6


In [10]:
final_df['isHoliday'] = ((pd.notna(holidays_local_merged['type_y']) |(pd.notna(holidays_regional_merged['type_y'])))|(pd.notna(holidays_national_merged['type_y']))).values
final_df['isHoliday']

id
3000888    False
3000889    False
3000890    False
3000891    False
3000892    False
           ...  
3029395    False
3029396    False
3029397    False
3029398    False
3029399    False
Name: isHoliday, Length: 28512, dtype: bool

In [11]:
final_df['dayofyear'] = pd.DatetimeIndex(pd.to_datetime(final_df['date'])).day_of_year.values
final_df['weekofyear'] = pd.DatetimeIndex(pd.to_datetime(final_df['date'])).isocalendar().week.values
final_df['weekday'] = pd.DatetimeIndex(pd.to_datetime(final_df['date'])).weekday.values

In [15]:
final_df['dcoilwtico'] = final_df['dcoilwtico'].fillna(method='bfill')

In [17]:
final_df_preprocessed = final_df.copy()

ordinal_columns = ['family','city','state','type','isHoliday']
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
final_df_preprocessed[ordinal_columns]=enc.fit_transform(final_df_preprocessed [ordinal_columns])

In [18]:
final_df_preprocessed.to_parquet('test_preprocessed.parquet')

In [19]:
final_df_preprocessed

,date,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster,isHoliday,dayofyear,weekofyear,weekday
id,,,,,,,,,,,,,
3000888,2017-08-16,1,0.0,0,46.80,18.0,12.0,3.0,13,0.0,228,33,2
3000889,2017-08-16,1,1.0,0,46.80,18.0,12.0,3.0,13,0.0,228,33,2
3000890,2017-08-16,1,2.0,2,46.80,18.0,12.0,3.0,13,0.0,228,33,2
3000891,2017-08-16,1,3.0,20,46.80,18.0,12.0,3.0,13,0.0,228,33,2
3000892,2017-08-16,1,4.0,0,46.80,18.0,12.0,3.0,13,0.0,228,33,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,2017-08-31,9,28.0,1,47.26,18.0,12.0,1.0,6,0.0,243,35,3
3029396,2017-08-31,9,29.0,0,47.26,18.0,12.0,1.0,6,0.0,243,35,3
3029397,2017-08-31,9,30.0,1,47.26,18.0,12.0,1.0,6,0.0,243,35,3
